# Summarization

<pre>
    Summarization in Langchain
        1. Stuff Document Chain
        2. Map Reduce Chain -> Summarizes bigger chunks -> summarises the summary
            a. Single Prompt
            b. Multiple Prompt  
        3. Refine Chain
</pre>

In [33]:
# Importing Libraries

from dotenv import load_dotenv
load_dotenv()
import os

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

from langchain_groq import ChatGroq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser

from langchain_community.document_loaders import WikipediaLoader

In [40]:
# Creating Template
template = """
Write a concise and short summary of the text : '{text}'
"""

prompt = PromptTemplate(input_variables=['text'], template=template)

llm = ChatGroq(model = "gemma2-9b-it")

In [35]:
 # Creating documents using Wikipedia Loader and then splitting the documents in chunks
docs = WikipediaLoader("Chelsea FC", load_max_docs=1, doc_content_chars_max=3000).load_and_split(RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 30))

# Stuff Document Chain
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm, chain_type='stuff', prompt = prompt, verbose= False)
stuff_summary = chain.run(docs)
stuff_summary

'Here is a concise and short summary of the text:\n\nChelsea Football Club is a professional football club based in London, England, founded in 1905. They have won six top-flight league titles, eight FA Cups, and several international trophies, including the UEFA Champions League, UEFA Europa League, and FIFA Club World Cup. The club has a rivalry with fellow London teams Arsenal, Tottenham Hotspur, and Fulham, and has been owned by BlueCo since 2022.'

## Map Reduce
For very large datasets - Summary of summaries of smaller chunks

In [36]:
 # Creating documents using Wikipedia Loader and then splitting the documents in chunks
docs1 = WikipediaLoader("Chelsea FC", load_max_docs=10, doc_content_chars_max=3000).load_and_split(RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 30))

In [41]:
prompt = PromptTemplate.from_template("Generate a concise summary of the following text : {text} ")

final_prompt_template = """
Provide the final summary of each document with the key points. 
Create a title of the summary. 
Return the summary with a title and all the major summarized points with an index starting from 1.

Here is the text : {text}
"""
final_prompt = PromptTemplate(input_variables = ['text'], template = final_prompt_template)

In [ ]:
map_reduce_summary_chain = load_summarize_chain(llm, 
                                                chain_type='map_reduce',
                                                map_prompt = prompt, # Combine the summaries of each document
                                                combine_prompt = final_prompt, # Generate summary of summaries
                                                verbose=False)
mr_summary = map_reduce_summary_chain.run(docs1)
mr_summary

"Here are the summaries with titles and key points:\n\n**Chelsea Football Club: A Century of Success**\n\nThis summary highlights the key milestones and achievements of Chelsea Football Club throughout its history.\n\n1. **Founding and Early Years:** Established in 1905, Chelsea quickly gained traction in English football, experiencing periods of both success and fluctuation.\n2. **Rise to Dominance:** Under manager Ted Drake, Chelsea clinched its first league title in 1955, marking a turning point in its journey to becoming a top club.\n3. **Modern Era Triumphs:** Chelsea's modern era has been defined by significant success, winning numerous Premier League titles, FA Cups, and major European trophies, including two Champions League titles.\n4. **International Recognition:** Chelsea holds the distinction of being the only London club to have won both the Champions League and Club World Cup.\n5. **Appearance Records:** Legendary players like Ron Harris, John Hollins, John Terry, and Fra

# Refine Summary
Get rolling summary - Summary of chunk gets passed as a reference to next chunk 

In [45]:
refine_summary_chain = load_summarize_chain(llm,
                                            chain_type='refine',
                                            verbose=False)
refine_summary = refine_summary_chain.run(docs)
refine_summary

'Chelsea Football Club, founded on 10 March 1905 at The Rising Sun pub (now The Butcher\'s Hook) in London, plays in the Premier League, the top tier of English football.  Several names were considered for the new club, including Kensington FC, Stamford Bridge FC and London FC, before settling on "Chelsea" after the adjacent borough to avoid confusion with the existing Fulham team. Their home ground is Stamford Bridge, which was originally an athletics stadium acquired by Gus Mears in 1904.  Mears initially planned to lease the stadium to Fulham F.C., but that proposal was turned down.  The club was elected to the Football League shortly after its founding.  \n\nIn their early years, Chelsea gained promotion to the First Division in their second season but quickly yo-yoed between the top two tiers.  They were known for signing star players, which contributed to their early success and attracted large crowds, even having the highest average attendance in English football in ten separate